# NLP Assignment 2: Language Modelling

## Team Members:
### Shrenik Ganguli - CS23MTECH14014
### Trishita Saha - CS23MTECH14016

In [1]:
# Importing necessary packages
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
#from nltk import bigrams, FreqDist, MLEProbDist, KneserNeyProbDist, ConditionalFreqDist, ConditionalProbDist
from nltk.lm.models import MLE, KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends, flatten
from nltk.lm.smoothing import KneserNey
import random
from nltk.util import pad_sequence
from nltk.stem import WordNetLemmatizer
import re

In [2]:
# Reading dataset
with open('Datasets/Harry_Potter_all_books_preprocessed.txt', 'r', encoding='utf-8') as file:
    data = file.read()

# Since fullstops('.') are not at the correct position making changes to the data
data = re.sub(r'\s\.([A-Z])', r'. \1', data)
print(data[:500])

THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much. They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense. Mr Dursley was the director of a firm called Grunnings which made drills. He was a big beefy man with hardly any neck although he did have a very large mustache. Mrs Dursley was thin and blonde and had nearly twice the usual amo


### 1. Preprocessing and tokenization of dataset

In [3]:
sentences = sent_tokenize(data)

# Preprocess each sentence
preprocessed_data = []
stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

word_count = 0

for sentence in sentences:
    # Tokenize into words
    words = word_tokenize(sentence)
    
    word_count += len(words)
    
    if word_count > 10000:
        break
    
    # Apply preprocessing steps
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word not in stop_words]
    
    # Append the preprocessed words to the list
    preprocessed_data.append(words)

### 2. Fitting two bigram language models on the text: MLE and Kneser-Ney discounting.

In [4]:
n = 2 # for bigrams
mle_train_data, mle_vocab = padded_everygram_pipeline(n, preprocessed_data)

mle = MLE(n)
mle.fit(mle_train_data, mle_vocab)

kn_train_data, kn_vocab = padded_everygram_pipeline(n, preprocessed_data)

# Create and train the Kneser-Ney interpolated model
kn = KneserNeyInterpolated(n, discount=0.1)
kn.fit(kn_train_data, kn_vocab)

### 3. Using the beginning words 1. ”Harry Potter” and 2. ”Dumbledore” to generate text using both the language models

In [5]:
seed_phrase = ['Harry', 'Potter']
seed_context = list(map(str.lower, seed_phrase))

# Add the seed context to the beginning of the generated sequence
generated_sequence = mle.generate(20, text_seed = seed_context, random_seed=7)
generated_sequence = [word for word in generated_sequence if word not in ['<s>', '</s>']]
full_generated_text = seed_context + generated_sequence
print('The first sequence generated with MLE and Harry Potter as beginning words is:')
print(' '.join(full_generated_text))
print("\n")

seed_word = 'Dumbledore'
seed_context2 = [seed_word.lower()]

generated_sequence = mle.generate(20, text_seed = seed_context2, random_seed=42)
generated_sequence = [word for word in generated_sequence if word not in ['<s>', '</s>']]
full_generated_text = seed_context2 + generated_sequence
print('The second sequence generated with MLE and Dumbledore as beginning word is:')
print(' '.join(full_generated_text))
print("\n\n")


generated_sequence = kn.generate(20, text_seed=seed_context, random_seed=7)
generated_sequence = [word for word in generated_sequence if word not in ['<s>', '</s>']]
full_generated_text = seed_context + generated_sequence
print('The first sequence generated with Kneser-Ney and Harry Potter as beginning words is:')
print(' '.join(full_generated_text))
print("\n")


generated_sequence = kn.generate(20, text_seed=seed_context2, random_seed=42)
generated_sequence = [word for word in generated_sequence if word not in ['<s>', '</s>']]
full_generated_text = seed_context2 + generated_sequence
print('The second sequence generated with Kneser-Ney and Dumbledore as beginning word is:')
print(' '.join(full_generated_text))

The first sequence generated with MLE and Harry Potter as beginning words is:
harry potter boy name inside envelope hide horrible scar his face bad news vernon opened front


The second sequence generated with MLE and Dumbledore as beginning word is:
dumbledore she cant blame hed said motorcycle said aunt petunia back outside door mysterious people celebrating i trying disturb



The first sequence generated with Kneser-Ney and Harry Potter as beginning words is:
harry potter boy parking lot mind left headlight swelled roar looked though didnt improve mood tabby cat mirror


The second sequence generated with Kneser-Ney and Dumbledore as beginning word is:
dumbledore said dudley except stupid people way harry corner privet drive parent died much time


In [6]:
print(len(mle.vocab),mle.counts[['harry']]['potter'])

1695 7


In [7]:
print(len(kn.vocab),kn.counts[['harry']]['potter'])

1695 7


### 4. Function to find the top k most probable words given some context

In [8]:
def top_k_words(model, context, k):
    
    
    #context = tuple(word.lower() for word in context if word.lower() in model.vocab)
    context = (context[-1].lower(),) if context and context[-1].lower() in model.vocab else ()
    
    # Use model.vocab for vocabulary
    vocab = set(model.vocab)
    
    # Filter out words not in the vocabulary
    context = tuple(word for word in context if word in vocab)
    
    context = context

    # Get the conditional probabilities for the next word
    cond_prob_dist = model.context_counts(context)

    # Sort the words based on their probabilities
    sorted_words = sorted(cond_prob_dist, key=cond_prob_dist.get, reverse=True)
    
    return sorted_words[:k]

beam_search_context = ['Harry','Potter',]
top_k = 10
top_words = top_k_words(mle, beam_search_context, top_k)
print(f'Top {top_k} words for the context "{beam_search_context}": {top_words}')

Top 10 words for the context "['Harry', 'Potter']": ['</s>', 'son', 'mr', 'arrived', 'small', 'away', 'thats', 'wasnt', 'if', 'involved']


### 5.Implementing Beam search

In [9]:
def beam_search_top_k(model, start_context, k, depth):
    # Initialize the beam with the start context
    beam = [(start_context, 1.0)]

    # Keep track of the top contexts during the iterations
    top_contexts = []

    for _ in range(depth):
        new_beam = []

        for context, score in beam:
            # Generate all possible next words
            next_words = set(model.vocab) - set(context)
            #next_words = top_k_words(model, context, k)
            #print(next_words)
            for next_word in next_words:
                # Calculate the score for the new context
                new_context = context + (next_word,)
                new_score = score * model.score(new_context[-1], new_context[:-1])

                # Add the new context and score to the beam
                new_beam.append((new_context, new_score))

        # Sort the new beam based on scores in descending order
        new_beam = sorted(new_beam, key=lambda x: x[1], reverse=True)

        # Keep only the top k contexts
        beam = new_beam[:k]

        # Extend the top_contexts list with the current top k contexts
        top_contexts.extend(beam)

    # Sort the top contexts based on scores in descending order and return the top 5
    top_contexts = sorted(top_contexts, key=lambda x: x[1], reverse=True)[:5]
    return [context for context, _ in top_contexts]

### 6.Repeating part 3 using Beam Search

In [12]:
start_context = ('Dumbledore',)
beam_width = 2
search_depth = 10
best_sequence_mle = beam_search_top_k(mle, start_context, beam_width, search_depth)
best_sequence_kn = beam_search_top_k(kn, start_context, beam_width, search_depth)

print("Top 5 sequences with MLE:")
for i, sequence in enumerate(best_sequence_mle):
    print(f"\nSequence {i+1}")
    
    for word in sequence:
        print(word,end=" ")
        
print("\n\nTop 5 sequences with Kneser Ney:")
for i, sequence in enumerate(best_sequence_kn):
    print(f"\nSequence {i+1}")
    for word in sequence:
        print(word,end=" ")

Top 5 sequences with MLE

Sequence 1
Dumbledore thinking 
Sequence 2
Dumbledore uncle 
Sequence 3
Dumbledore thinking uncle 
Sequence 4
Dumbledore thinking swung 
Sequence 5
Dumbledore thinking uncle swung 

Top 5 sequences with Kneser Ney

Sequence 1
Dumbledore </s> 
Sequence 2
Dumbledore harry 
Sequence 3
Dumbledore harry </s> 
Sequence 4
Dumbledore </s> harry 
Sequence 5
Dumbledore </s> harry potter 